In [95]:
import numpy as np

def sigmoid(x):
    return 1/(1+np.exp(-x))

def crossEntropyS(X, Y, w, b):
	# X.shape = (S,N) Y.shape = (S), W.shape = (N) 
    v = sigmoid(np.dot(X,w)+b)
    p = (1-Y) + (2*Y-1)*v
    ce = -np.mean(np.log(p))
    return ce

def gradCrossEntropyS(X,Y,w,b):
    g = Y - sigmoid(np.dot(X,w)+b)
    db = -np.mean(g)
    dw = -np.mean( g[:,np.newaxis] * X,axis=0)
    return dw, db   

In [196]:
# helped by:
# https://www.kaggle.com/code/anubhavgoyal10/one-hidden-layer-neural-network-from-scratch

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# hidden_layers = int(input("Брой неврони от скрития слой"))
# learn_repeat = int(input("Брой повторения на обучаването"))
k = 10
learn_repeat = 50000
learning_rate = 0.5
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y_and = np.array([0,0,0,1])
y_or = np.array([0,1,1,1])
y_xor = np.array([0,1,1,0])

y = y_xor

# -------------------TEST-------------------
# from sklearn.datasets import make_gaussian_quantiles
# N = 500
# X, y = make_gaussian_quantiles(mean=None, cov=0.5, n_samples = N, 
#                                              n_features = 2, n_classes=2, shuffle = True, 
#                                              random_state = 42)
# -------------------TEST-------------------



Wh = np.random.uniform(-1, 1, (X.shape[1], k))
bh = np.random.uniform(-1, 1, (k,))

Wo = np.random.uniform(-1, 1, (k, ))
bo = np.random.uniform(-1, 1)

for i in range(learn_repeat):
    if i%1000 == 0:
        print(f"\rRepetition: {i}/{learn_repeat}", end='', flush=True)
    q = np.dot(X, Wh) + bh
    # r = np.tanh(q)
    r = sigmoid(q)
    s = np.dot(r, Wo) + bo
    sc = sigmoid(s)
    p = (1-y) + (2*y-1)*sc
    error = -np.mean(np.log(p))
    # print(p)
    # print(error)
    
    dWo, dbo = gradCrossEntropyS(r,y,Wo,bo)
    ds = -np.mean(y - sc)
    dt = ds
    dr = np.outer(np.ones(k), ds).T * np.outer(np.ones(X.shape[0]), Wo)
    # dq = dr * (1 - np.power(np.tanh(q),2))
    dq = dr * sigmoid_derivative(q)
    dbh = np.mean(dq, axis=0)
    dWh = X.T @ dq # should there be a division by k?
    
    Wh = Wh - learning_rate * dWh
    Wo = Wo - learning_rate * dWo
    bh = bh - learning_rate * dbh
    bo = bo - learning_rate * dbo
# print(sc)
print("\nerror: " + str(error))
print(sc.round(decimals=2))
print(np.sum(p > 0.5))
print(X.shape[0])

Repetition: 49000/50000
error: 0.39270570526688353
[0.03 0.71 0.69 0.57]
3
4
